# SENNA для задачи POS-tagging

SENNA (Semantic/syntactic Extraction using a Neural Network Architecture) – архитектура нейронной сети, позволяющая достигнуть state-of-the-art результатов в нескольких задачах обработки текстов. 


#### Задача POS-tagging
ставится как задача многоклассовой классификации: 

* $T$ - количество различных тегов частей речи (каждое слово $w$ относится к одному из $T$ классов)
* для каждого слова из train формируется вектор признаков 
* NN обучается по всем векторам признаков каждого слова из train 

#### Подход к решению 
представлен в https://arxiv.org/pdf/1103.0398.pdf( Window approach network, раздел 3.3.1):
1. Каждое слово представляется эмбеддингом размерности $d$;
2. Для каждого слова формируется окно длины $k$ из $(k-1)/2$ соседних слов слева от данного слова  и $(k-1)/2$ соседних слов справа от данного слова, $k$ – нечетное. (Если для слова невозможно найти соседние слова, используется padding.)
3. Для каждого слова формируется вектор признаков, состоящий из конкатенированных эмбеддингов слов из левого окна, данного слова и слов из правого окна. Итоговая размерность вектора признаков – $d \times k$. Этот вектор подается на вход нейронной сети;
4. Обучается нейронная сеть, имеющая один скрытый слой с $n_h$ нейроннами и нелинейной функцией активации $\theta$;
5. На выходном слое нейронной сети решается задача классификации на |T| классов. 


Открытый корпус: https://github.com/dialogue-evaluation/morphoRuEval-2017/blob/master/OpenCorpora_Texts.rar

Предобученные эмбеддинги Facebook: https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.ru.vec

In [ ]:
! wget https://www.dropbox.com/s/n5pgf9nu50jvwra/unamb_sent_14_6.conllu
! wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.ru.vec

In [ ]:
! pip install nltk
! pip install tqdm
! pip install keras

### 1. Составляем обучающую выборку 

- считываем выборку 
- делим на train, test по предложениям
- каждое предложение внутри каждого из множества разделям на слова (оставляем структуру предложения в виде list, потому что нам потребуется контекст: слова слева и справа)

In [1]:
path = 'unamb_sent_14_6.conllu'
project_path = '/home/nkozlovskaya/nlp_course'

In [2]:
from nltk.corpus.reader import ConllCorpusReader
pos_corpus = ConllCorpusReader(project_path, fileids = path, 
                               columntypes = ['ignore', 'words', 'ignore', 'pos', 'chunk'])
sents = list(pos_corpus.iob_sents())

In [3]:
pos_tags = set([pos for text in sents for word, pos, chunk in text])

In [4]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(sents, test_size=0.25)
sent_train = [[word[0].lower() for word in sent]for sent in train]
label_train = [[word[1] for word in sent] for sent in train]
sent_test = [[word[0].lower() for word in sent]for sent in test]
label_test = [[word[1] for word in sent] for sent in test]

### 2. Считываем эмбеддинги

Далее будем подавать эмбеддинги на вход Embedding layer в поле weights (матрица). Матрица train при этом должна быть integer-encoded (слову соответствует индекс), т.е. строка матрицы - контекст слова, для которого есть POS-метка.

- будем хранить не все эмбеддинги, а только для слов, которые встречаются в train и test: надо сохранить сами эмбеддинги в матрицу (word_embeddings), и запомнить соответствие слово <-> индекс в матрице (word_2_idx)
- будьте внимательны с кодировкой .de(en)code('utf-8')
- не забудем о PADDING, UNKNOWN (для некоторых слов не существует контекста, в этом случае эмбеддинг будет из нулей; для некоторых слов не найдется предобученного эмбеддинга, создадим для таких слов эмбеддинг np.random.uniform)

In [5]:
# чтобы знать, какие слова есть
words = set()

for sent_set in [sent_train, sent_test]:
    for sentence in sent_set:
        for token in sentence:
            words.add(token.lower())

In [6]:
from tqdm import tqdm
import numpy as np

word_2_idx = {}
word_embeddings = []
with open('wiki.ru.vec', 'rb') as f :
    for line in tqdm(f):
        values = line.split()
        if len(values) != 301:
            continue
            
        if len(word_2_idx) == 0:
            word_2_idx["padding"] = len(word_2_idx)
            vector = np.zeros(len(values)-1) 
            word_embeddings.append(vector)

            word_2_idx["unknown"] = len(word_2_idx)
            vector = np.random.uniform(-0.25, 0.25, len(values)-1)
            word_embeddings.append(vector)
            values = line.split()
        if  values[0].lower().decode('utf-8') in words:
            vector = np.asarray(values[1:], dtype='float32')
            word_embeddings.append(vector)
            word_2_idx[values[0].lower()] = len(word_2_idx)
            
            
word_embeddings = np.array(word_embeddings)

1888424it [00:38, 48518.50it/s]


In [7]:
word_embeddings.shape, len(word_2_idx), len(words)

((70789, 300), 70789, 79791)

### 3. Составляем train

- сформируем окно для каждого слова размера $k$;
- закодируем каждое слово из контекста индексом, соответсвующим этому слову в матрице эмбеддингов
- не забываем про padding, unknown

### 4. Составляем test
- кодируем каждый label индексом

In [21]:
WINDOWSIZE = 5
UNKNOWN_IDX = word_2_idx['unknown']
PADDING_IDX = word_2_idx['padding']

In [22]:
def get_context(tgt_word_idx, sentence, windowsize):
    context = []  
    for word_position in range(tgt_word_idx - windowsize, tgt_word_idx + windowsize+1):
        if word_position < 0 or word_position >= len(sentence):
            context.append("padding")
            continue
        word = sentence[word_position]
        context.append(word)   
    return context


# сюда будем записывать не сами слова, а индекс эмбеддинга
X_train = []

for sentence in sent_train:
    for tgt_word_idx in range(len(sentence)):
        tgt_word_context = get_context(tgt_word_idx, sentence, WINDOWSIZE/2)
        X_train.append([word_2_idx.get(word.encode('utf-8'), UNKNOWN_IDX) for word in tgt_word_context])

        
label_2_idx = {}
for label in pos_tags:
    label_2_idx[label] = len(label_2_idx)
y_train = []
for el in label_train:
    y_train.extend([label_2_idx.get(label) for label in el])
    
X_train = np.array(X_train)
y_train = np.array(y_train)

### 5. Обучаем NN


In [23]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers import Embedding

n_in = X_train.shape[1] # windowsize
n_out = len(label_2_idx) # num of labels

# trainable=False using fixed embeddings for a text input
words_input = Input(shape=(n_in,), dtype='int32', name='words_input')
words = Embedding(input_dim=word_embeddings.shape[0], output_dim=word_embeddings.shape[1], 
                  weights=[word_embeddings], trainable=False)(words_input)
words = Flatten()(words)

output = Dense(64, activation='tanh')(words)
output = Dense(n_out, activation='softmax')(output)

model = Model(input=[words_input], output=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam')
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
words_input (InputLayer)         (None, 5)             0                                            
____________________________________________________________________________________________________
embedding_5 (Embedding)          (None, 5, 300)        21236700    words_input[0][0]                
____________________________________________________________________________________________________
flatten_5 (Flatten)              (None, 1500)          0           embedding_5[0][0]                
____________________________________________________________________________________________________
dense_9 (Dense)                  (None, 64)            96064       flatten_5[0][0]                  
___________________________________________________________________________________________

In [28]:
model.fit(X_train, y_train, nb_epoch= 2, batch_size = 32,  validation_split = 0.1)

Train on 308996 samples, validate on 34333 samples
Epoch 1/2
308996/308996 [==============================] - 35s - loss: 0.1114 - val_loss: 0.1808
Epoch 2/2
308996/308996 [==============================] - 35s - loss: 0.0977 - val_loss: 0.1884


### 6. Делаем предсказание для test
- составляем test
- применяем модель
- смотрим на качество

In [29]:
X_test = []

for sentence in sent_test:
    for tgt_word_idx in range(len(sentence)):
        tgt_word_context = get_context(tgt_word_idx, sentence, WINDOWSIZE/2)
        X_test.append([word_2_idx.get(word.encode('utf-8'), UNKNOWN_IDX) for word in tgt_word_context])
        
y_test = []
for el in label_test:
    y_test.extend([label_2_idx.get(label) for label in el])
    
X_test = np.array(X_test)
y_test = np.array(y_test)

In [30]:
idx_2_label = {v: k for k, v in label_2_idx.items()}
pred = model.predict(X_test)
t = np.array([idx_2_label[i] for i in y_test])
p = np.array([idx_2_label[i] for i in np.argmax(pred, axis=1)])

In [31]:
from sklearn.metrics import classification_report
print np.mean(t==p)
print classification_report(t, p)

0.9454636161534826
             precision    recall  f1-score   support

        ADJ       0.96      0.91      0.94     11869
        ADP       0.99      0.99      0.99     10726
        ADV       0.88      0.89      0.89      3374
       CONJ       0.96      0.98      0.97      5532
        DET       0.96      0.95      0.96      3112
       INTJ       0.76      0.50      0.60       122
       NOUN       0.96      0.97      0.97     30494
        NUM       0.87      0.79      0.83      2605
       PART       0.95      0.89      0.92      2207
       PRON       0.97      0.95      0.96      2221
      PROPN       0.83      0.90      0.86      3690
      PUNCT       0.96      0.99      0.98     22684
       VERB       0.96      0.97      0.97     10414
          X       0.71      0.69      0.70      5204

avg / total       0.95      0.95      0.95    114254

